In [1]:
# =======================================================
# IMPORTS
# =======================================================
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# =======================================================
# LOAD DATA
# =======================================================
train = pd.read_csv("/kaggle/input/mock-test-2-mse-2/train.csv")
test  = pd.read_csv("/kaggle/input/mock-test-2-mse-2/test.csv")

# =======================================================
# CHECK MISSING VALUES
# =======================================================
print(train.isna().sum())
print(test.isna().sum())

# =======================================================
# HANDLE MISSING VALUES
# =======================================================

# Categorical columns EXCEPT Status
cat_cols = train.select_dtypes(include=['object']).columns.tolist()
if "Status" in cat_cols:
    cat_cols.remove("Status")

# Numerical columns
num_cols = train.select_dtypes(include=['int64', 'float64']).columns

# Fill categorical with mode
for col in cat_cols:
    train[col] = train[col].fillna(train[col].mode()[0])
    if col in test.columns:
        test[col]  = test[col].fillna(test[col].mode()[0])

# Fill numeric with mean
for col in num_cols:
    train[col] = train[col].fillna(train[col].mean())
    if col in test.columns:
        test[col]  = test[col].fillna(test[col].mean())

# =======================================================
# SAFE LABEL ENCODING (TRAIN + TEST TOGETHER)
# =======================================================
le_dict = {}

for col in cat_cols:
    le = LabelEncoder()

    combined = pd.concat([train[col].astype(str),
                          test[col].astype(str)], axis=0)

    le.fit(combined)

    train[col] = le.transform(train[col].astype(str))
    test[col]  = le.transform(test[col].astype(str))

    le_dict[col] = le

# =======================================================
# SPLIT FEATURES AND TARGET
# =======================================================
X = train.drop(columns=["Status"])
y = train["Status"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# =======================================================
# KNN MODEL
# =======================================================
model = KNeighborsClassifier(
    n_neighbors=30,
    weights="distance"
)

model.fit(X_train, y_train)

# =======================================================
# VALIDATION
# =======================================================
y_pred = model.predict(X_test)
print("Validation Accuracy:", accuracy_score(y_test, y_pred))

# =======================================================
# FINAL PREDICTION ON TEST
# =======================================================
final_pred_proba = model.predict_proba(test)

# =======================================================
# CREATE SUBMISSION FILE
# =======================================================
submission = pd.DataFrame({
    "id": test["id"],
    "Status_C":  final_pred_proba[:, 0],
    "Status_CL": final_pred_proba[:, 1],
    "Status_D":  final_pred_proba[:, 2]
})

submission.to_csv("/kaggle/working/Answer.csv", index=False)
submission.head()

Loading data...


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ai-201-b-mse-2-ai-d/train.csv'